Code for turn optimization 
(with four-wheeled skate, constraints not fixed for four-wheels)

In [2]:
import numpy as np
from functools import partial

from pydrake.all import (
    AddDefaultVisualization,
    DiscreteContactApproximation,
    PidController,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    AddMultibodyPlantSceneGraph,
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    JacobianWrtVariable,
    JointIndex,
    MathematicalProgram,
    MeshcatVisualizer,
    OrientationConstraint,
    Parser,
    PiecewisePolynomial,
    PositionConstraint,
    RotationMatrix,
    SnoptSolver,
    Solve,
    eq,
    namedview,
)

from underactuated import ConfigureParser, running_as_notebook
from underactuated.multibody import MakePidStateProjectionMatrix    

# import os
# print(os.path.abspath(os.getcwd()))

In [3]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [5]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
# NOTE:
# share with Savva (savva@mit.edu)

def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(
        ExtractGradient(a), ExtractGradient(b)
    )


def gait_optimization(gait="walking_trot"):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    ConfigureParser(parser)
    (spot,) = parser.AddModels("spot4wd.dmd.yaml") #changed to spotblades
    plant.Finalize()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    diagram.ForcedPublish(context)

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")
    PositionView = namedview(
        "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
    )
    VelocityView = namedview(
        "Velocities",
        plant.GetVelocityNames(spot, always_add_suffix=False),
    )

    mu = 0.9  
    total_mass = plant.CalcTotalMass(plant_context, [spot])
    gravity = plant.gravity_field().gravity_vector()

    # nq = 30
    # foot_frame = [
    #     plant.GetFrameByName("first_front_right_wheel"),
    #     plant.GetFrameByName("first_front_left_wheel"),
    #     plant.GetFrameByName("first_rear_right_wheel"),
    #     plant.GetFrameByName("first_rear_left_wheel")]
    
    # second_foot_frame = [
    #     plant.GetFrameByName("second_front_right_wheel"),
    #     plant.GetFrameByName("second_front_left_wheel"),
    #     plant.GetFrameByName("second_rear_right_wheel"),
    #     plant.GetFrameByName("second_rear_left_wheel")
    # ]

    foot_frame = [
        plant.GetFrameByName("first_front_right_wheel"),
        plant.GetFrameByName("first_front_left_wheel"),
        plant.GetFrameByName("first_rear_right_wheel"),
        plant.GetFrameByName("first_rear_left_wheel"),
        plant.GetFrameByName("second_front_right_wheel"),
        plant.GetFrameByName("second_front_left_wheel"),
        plant.GetFrameByName("second_rear_right_wheel"),
        plant.GetFrameByName("second_rear_left_wheel"),
        plant.GetFrameByName("third_front_right_wheel"),
        plant.GetFrameByName("third_front_left_wheel"),
        plant.GetFrameByName("third_rear_right_wheel"),
        plant.GetFrameByName("third_rear_left_wheel"),
        plant.GetFrameByName("fourth_front_right_wheel"),
        plant.GetFrameByName("fourth_front_left_wheel"),
        plant.GetFrameByName("fourth_rear_right_wheel"),
        plant.GetFrameByName("fourth_rear_left_wheel")
    ]
    if gait == "skate":
        N = 21
        in_stance = np.zeros((4, N))
        in_stance[0, 0:N] = 1 #for skate, using 0 as on ground and 1 as off ground
        in_stance[1, 0:N] = 1
        in_stance[2, 4:N] = 1 #only left leg lifted for 0:15 in stance
        in_stance[3, 0:18] = 1 #only right leg lifted for 26:N in stance
        speed = 0.7
        is_laterally_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    elif gait == "stop":
        N = 21
        in_stance = np.ones((4, N))
        speed = 0.02
        is_laterally_symmetric = True
        check_self_collision = True
        stride_length = 0.3
    elif gait == "left":
        N = 21
        in_stance = np.ones((4, N))
        in_stance[2, 0:10] = 0
        in_stance[0, 11:N] = 0
        speed = 0.5
        is_laterally_symmetric = False
        check_self_collision = True
        stride_length = 0.55
    elif gait=="swimmer":
        N = 21
        in_stance = np.ones((16, N))
        speed = 0.5
        is_laterally_symmetric = False
        check_self_collision = True
        stride_length = 4
        direction = "left"
        #goal angles = np.range()
    else:
        raise RuntimeError("Unknown gait.")


    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0
    T = 4
    prog = MathematicalProgram()

    # Time steps
    h = prog.NewContinuousVariables(N - 1, "h")
    prog.AddBoundingBoxConstraint(0.5 * T / N, 2.0 * T / N, h)
    prog.AddLinearConstraint(sum(h) >= 0.1 * T)
    prog.AddLinearConstraint(sum(h) <= 2.7 * T)

    # Create one context per time step (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    # Joint costs
    q_cost = PositionView([1] * nq)
    v_cost = VelocityView([1] * nv)
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0
    
    q_cost.front_left_hip_x = 0.7
    q_cost.front_right_hip_x = 0.7
    q_cost.rear_left_hip_x = 0.7
    q_cost.rear_right_hip_x = 0.7
    v_cost.body_vx = 0
    v_cost.body_wx = 0
    v_cost.body_wy = 0
    v_cost.body_wz = 0

    q_cost.front_right_leg_to_wheel = 0.1
    q_cost.second_front_right_leg_to_wheel = 0.1
    q_cost.front_left_leg_to_wheel = 0.1
    q_cost.second_front_left_leg_to_wheel = 0.1
    q_cost.rear_right_leg_to_wheel = 0.1
    q_cost.second_rear_right_leg_to_wheel = 0.1
    q_cost.rear_right_leg_to_wheel = 0.1
    q_cost.second_rear_left_leg_to_wheel = 0.1
    q_cost.front_right_leg_to_platform = 0.1
    q_cost.front_left_leg_to_platform = 0.1
    q_cost.rear_right_leg_to_platform = 0.1
    q_cost.rear_left_leg_to_platform = 0.1

    v_cost.front_right_leg_to_wheel = 0
    v_cost.second_front_right_leg_to_wheel = 0
    v_cost.front_left_leg_to_wheel = 0
    v_cost.second_front_left_leg_to_wheel = 0
    v_cost.rear_right_leg_to_wheel = 0
    v_cost.second_rear_right_leg_to_wheel = 0
    v_cost.rear_right_leg_to_wheel = 0
    v_cost.second_rear_left_leg_to_wheel = 0
    v_cost.front_right_leg_to_platform = 0
    v_cost.front_left_leg_to_platform = 0
    v_cost.rear_right_leg_to_platform = 0
    v_cost.rear_left_leg_to_platform = 0

    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(
            plant.GetPositionLowerLimits(),
            plant.GetPositionUpperLimits(),
            q[:, n],
        )
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(
            plant.GetVelocityLowerLimits(),
            plant.GetVelocityUpperLimits(),
            v[:, n],
        )
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:, n], prog)
        # Body orientation
        # print(f"body frame at step {n}: {body_frame}")
        if n == 0:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    0.1,
                    context[n],
                ),
                q[:, n],
            )
        else:
            prog.AddConstraint(
                OrientationConstraint(
                    plant,
                    body_frame,
                    RotationMatrix(),
                    plant.world_frame(),
                    RotationMatrix(),
                    1,
                    context[n],
                ),
                q[:, n],
            )
        # prog.AddConstraint(
        #     OrientationConstraint(
        #         plant,
        #         body_frame,
        #         RotationMatrix(),
        #         plant.world_frame(),
        #         RotationMatrix(),
        #         1.57,
        #         context[n],
        #     ),
        #     q[:, n],
        # )
    # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(
            q[:, n], q0
        )  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:, n])
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v[:, n])

    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1 + nq, 1 + nq + nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                q,
                ad_plant.GetPositions(ad_velocity_dynamics_context[context_index]),
            ):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(
                ad_velocity_dynamics_context[context_index], (qn - q) / h
            )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q) / h)
        return v - v_from_qdot

    for n in range(N - 1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n),
            lb=[0] * nv,
            ub=[0] * nv,
            vars=np.concatenate(([h[n]], q[:, n], v[:, n], q[:, n + 1])),
        )

    # Contact forces
    # weird friction cone
    # slant legs
    # dyanmics of motion for the blades
    contact_force = [ #TODO: edited
        prog.NewContinuousVariables(3, N - 1, f"foot{foot}_contact_force")
        for foot in range(16)
    ]
    for n in range(N - 1):
        for foot in range(16):
            # Linear friction cone
            prog.AddLinearConstraint(
                contact_force[foot][0, n] <= mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][0, n] <= mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                contact_force[foot][1, n] <= mu * contact_force[foot][2, n]
            )
            prog.AddLinearConstraint(
                -contact_force[foot][1, n] <= mu * contact_force[foot][2, n]
            )
            # normal force >=0, normal_force == 0 if not in_stance
            prog.AddBoundingBoxConstraint(
                0,
                in_stance[foot, n] * 4 * 9.81 * total_mass,
                contact_force[foot][2, n],
            )

    # Center of mass variables and constraints
    eps = 1e-3
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N - 1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2, 0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2, 0])
    # CoM height
    prog.AddBoundingBoxConstraint(0.45, np.inf, com[2, :])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0, :])
    # orientation of CoM

    for n in range(N-1):
        prog.AddLinearConstraint(comdot[0, n] >= 2)
    # CoM final x position
    # if is_laterally_symmetric:
    #     prog.AddBoundingBoxConstraint(
    #         stride_length / 2.0, stride_length / 2.0, com[0, -1]
    #     )
    # else:
    #     prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0, -1])
    # for n in range(N-1):
    #     prog.AddQuadraticCost(-2*comdot[:,n].T @ comdot[:,n])
    # CoM dynamics
    for n in range(N - 1):
        # Note: The original matlab implementation used backwards Euler (here and throughout),
        # which is a little more consistent with the LCP contact models.
        prog.AddConstraint(eq(com[:, n + 1], com[:, n] + h[n] * comdot[:, n]))
        prog.AddConstraint(eq(comdot[:, n + 1], comdot[:, n] + h[n] * comddot[:, n]))
        prog.AddConstraint(
            eq(
                total_mass * comddot[:, n],
                sum(contact_force[i][:, n] for i in range(16)) + total_mass * gravity, #HERE
            )
        )
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N - 1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3, N - 1)))

    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq + 3, nq + 6])
        contact_force = contact_force.reshape(3, 4, order="F")
        if isinstance(vars[0], AutoDiffXd):
            dq = ExtractGradient(q)
            q = ExtractValue(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(4): #HERE
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                Jq_WF = plant.CalcJacobianTranslationalVelocity(
                    context[context_index],
                    JacobianWrtVariable.kQDot,
                    foot_frame[i],
                    [0, 0, -0.03],
                    plant.world_frame(),
                    plant.world_frame(),
                )
                ad_p_WF = InitializeAutoDiff(p_WF, Jq_WF @ dq)
                torque = torque + np.cross(
                    ad_p_WF.reshape(3) - com, contact_force[:, i]
                )
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(4): #HERE
                p_WF = plant.CalcPointsPositions(
                    context[context_index],
                    foot_frame[i],
                    [0, 0, -0.03],
                    plant.world_frame(),
                )
                torque += np.cross(p_WF.reshape(3) - com, contact_force[:, i])
        return Hdot - torque

    for n in range(N - 1):
        prog.AddConstraint(eq(H[:, n + 1], H[:, n] + h[n] * Hdot[:, n]))
        Fn = np.concatenate([contact_force[i][:, n] for i in range(4)]) #HERE
        prog.AddConstraint(
            partial(angular_momentum_constraint, context_index=n),
            lb=np.zeros(3),
            ub=np.zeros(3),
            vars=np.concatenate((q[:, n], com[:, n], Hdot[:, n], Fn)),
        )

    # com == CenterOfMass(q), H = SpatialMomentumInWorldAboutPoint(q, v, com)
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]

    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq + nv, nq + nv + 3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(
                qv,
                ad_plant.GetPositionsAndVelocities(
                    com_constraint_context[context_index]
                ),
            ):
                ad_plant.SetPositionsAndVelocities(
                    com_constraint_context[context_index], qv
                )
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(
                com_constraint_context[context_index], [spot]
            )
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(
                com_constraint_context[context_index], [spot], com
            ).rotational()
        else:
            if not np.array_equal(
                qv, plant.GetPositionsAndVelocities(context[context_index])
            ):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(
                context[context_index], [spot]
            )
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(
                context[context_index], [spot], com
            ).rotational()
        return np.concatenate((com_q - com, H_qv - H))

    for n in range(N):
        prog.AddConstraint(
            partial(com_constraint, context_index=n),
            lb=np.zeros(6),
            ub=np.zeros(6),
            vars=np.concatenate((q[:, n], v[:, n], com[:, n], H[:, n])),
        )

    # TODO: Add collision constraints
    # Wheels cannot collide:

    class Trajectory:
        def __init__(self, coords, n_pts):
            self.pts = coords
            self.num_pts = len(self.pts)
            self.n_pts = n_pts
        
        def format_coords(self):
            return [np.asarray(coord) for coord in coords]

        def build_Q(self, weights=None):
            if not weights:
                return np.ones(2)
            return np.diag(weights)

        def gen_traj_constraints(self):
            Q = self.build_Q()
            crds = self.format_coords()
            for (n, c) in zip(self.n_pts, crds):
                if n == 20:
                    prog.AddBoundingBoxConstraint(c[0] - 0.1, c[0] + 0.1, com[0, n])
                    prog.AddBoundingBoxConstraint(c[1] - 0.1, c[1] + 0.1, com[1, n])
                else:
                    prog.AddQuadraticErrorCost(2*Q, c, com[:2, n])

        def gen_orientation_constraint(self):
            """cost on orientation change from overall direction"""
            fin = self.pts[-1]
            angle = np.arctan(fin[1]/fin[0])

    coords = [(4, 5)]
    n_pts = [20]
    for n in range(N):
        prog.AddQuadraticErrorCost(0.8*np.ones(2), np.asarray(coords[-1]), com[:2, n])
    traj_opter = Trajectory(coords, n_pts)
    traj_opter.gen_traj_constraints()
    prev_body_frame = 0
    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index + 1])):
            plant.SetPositions(context[context_index + 1], qn)
        p_WF = plant.CalcPointsPositions(
            context[context_index], frame, [0, 0, 0], plant.world_frame()
        )
        p_WF_n = plant.CalcPointsPositions(
            context[context_index + 1], frame, [0, 0, 0], plant.world_frame()
        )
        if isinstance(vars[0], AutoDiffXd):
            J_WF = plant.CalcJacobianTranslationalVelocity(
                context[context_index],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            J_WF_n = plant.CalcJacobianTranslationalVelocity(
                context[context_index + 1],
                JacobianWrtVariable.kQDot,
                frame,
                [0, 0, 0],
                plant.world_frame(),
                plant.world_frame(),
            )
            return InitializeAutoDiff(
                p_WF_n - p_WF,
                J_WF_n @ ExtractGradient(qn) - J_WF @ ExtractGradient(q),
            )
        else:
            return p_WF_n - p_WF

    for i in range(16):
        for n in range(N):
            if in_stance[i, n]:
                if i%2 == 0:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, -np.inf, 0],
                            [np.inf, -0.01, 0],
                            # [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    #SECOND WHEEL TRIAL RUN
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, -np.inf, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, -0, 0],
                    #         # [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )
                else:
                    prog.AddConstraint(
                        PositionConstraint(
                            plant,
                            plant.world_frame(),
                            # [np.inf, np.inf, 0],
                            [-np.inf, 0.01, 0],
                            # [-np.inf, -np.inf, 0],
                            [np.inf, np.inf, 0],
                            foot_frame[i],
                            #[0, 0, -0.4325],
                            [0, 0, -0.03],
                            context[n]),
                        q[:, n],
                    )
                    # prog.AddConstraint( #added
                    #     PositionConstraint(
                    #         plant,
                    #         plant.world_frame(),
                    #         [-np.inf, 0, 0],
                    #         # [-np.inf, -np.inf, 0],
                    #         # [np.inf, np.inf, 0],
                    #         [np.inf, np.inf, 0],
                    #         second_foot_frame[i],
                    #         #[0, 0, -0.4325],
                    #         [0, 0, -0.03],
                    #         context[n]),
                    #     q[:, n],
                    # )


    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, "Iterations Limits", 1e5 if running_as_notebook else 1)
    prog.SetSolverOption(
        snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
    )
    prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
    prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
    prog.SetSolverOption(snopt, "Superbasics limit", 2000)
    prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
    # prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    # TODO a few more costs/constraints from
    # from https://github.com/RobotLocomotion/LittleDog/blob/master/gaitOptimization.m

    result = Solve(prog)
    print(result.get_solver_id().name())
    print(result.is_success())  # We expect this to be false if iterations are limited.

    def HalfStrideToFullStride(a):
        b = PositionView(np.copy(a))

        b.body_y = -a.body_y
        # Mirror quaternion so that roll=-roll, yaw=-yaw
        b.body_qx = -a.body_qx
        b.body_qz = -a.body_qz

        b.front_left_hip_x = -a.front_right_hip_x
        b.front_right_hip_x = -a.front_left_hip_x
        b.rear_left_hip_x = -a.rear_right_hip_x
        b.rear_right_hip_x = -a.rear_left_hip_x

        b.front_left_hip_y = a.front_right_hip_y
        b.front_right_hip_y = a.front_left_hip_y
        b.rear_left_hip_y = a.rear_right_hip_y
        b.rear_right_hip_y = a.rear_left_hip_y

        b.front_left_knee = a.front_right_knee
        b.front_right_knee = a.front_left_knee
        b.rear_left_knee = a.rear_right_knee
        b.rear_right_knee = a.rear_left_knee

        return b

    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0], result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.StartRecording()
    num_strides = 1
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf * num_strides * (2.0 if is_laterally_symmetric else 1.0)
    for t in np.hstack((np.arange(t0, T, 1.0 / 32.0), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        # if is_laterally_symmetric:
        #     if stride % 2 == 1:
        #         qt = HalfStrideToFullStride(qt)
        #         qt.body_x += stride_length / 2.0
        #     stride = stride // 2
        # qt.body_x += stride * stride_length
        plant.SetPositions(plant_context, qt[:])
        diagram.ForcedPublish(context)

    visualizer.StopRecording()
    visualizer.PublishRecording()


# Try them all!  The last three could use a little tuning.
#gait_optimization("walking_trot")
#gait_optimization("skate")
gait_optimization("swimmer")
#gait_optimization("left")
# gait_optimization("running_trot")
# gait_optimization('rotary_gallop')
# gait_optimization('bound')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=545d651b-e7e0-4cf4-bfbe-97d6a3f869f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>